In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
def calc_outflow(inflowpath, dspath, epath, savepath):
    inflow = pd.read_csv(inflowpath, parse_dates=["date"]) # read in inflow (m3/s)
    E = pd.read_csv(epath, parse_dates=['time']) # read in evaporation (mm/d)

    if isinstance(dspath, str):
        df = pd.read_csv(dspath, parse_dates=['date'])
    else:
        df = dspath
    
    inflow = inflow[inflow['date']>=df['date'].iloc[0]]
    inflow = inflow[inflow['date']<=df['date'].iloc[-1]]
    inflow = inflow.set_index('date')

    inflow['streamflow'] = inflow['streamflow'] * (60*60*24) # convert from m3/s to m3/d

    E = E[E['time'] >= df['date'].iloc[0]]
    E = E[E['time'] <= df['date'].iloc[-1]]
    E = E.set_index('time')

    area = df[['corrected_area', 'date']].set_index('date').resample('1D').interpolate(method='linear')['corrected_area']

    E['OUT_EVAP'] = E['OUT_EVAP'] * (0.001 * area * 1000*1000)  # convert mm/d to m3/d. E in mm, area in km2

    last_date = df['date'][:-1]
    df = df.iloc[1:,:]
    
    df['last_date'] = last_date.values
    
    df['inflow_vol'] = df.apply(lambda row: inflow.loc[row['last_date']:row['date'], 'streamflow'].sum(), axis=1)  # accumulate inflow
    df['evap_vol'] = df.apply(lambda row: E.loc[(E.index > row['last_date'])&(E.index <= row['date']), 'OUT_EVAP'].sum(), axis=1) # accumulate evaporation
    df['outflow_vol'] = df['inflow_vol'] - df['dS'] - df['evap_vol']    # get outflow volume
    df['days_passed'] = (df['date'] - df['last_date']).dt.days
    df['outflow_rate'] = df['outflow_vol']/df['days_passed']   # m3/d
    df.loc[df['outflow_rate']<0, 'outflow_rate'] = 0

    df.to_csv(savepath, index=False)

In [ ]:
inflowpath = "../data/inflow-sirindhorn.csv"
evappath = "../data/evaporation-sirindhorn.csv"
delspath = "../data/dels-sirindhorn.csv"
savepath = "../data/outflow-sirindhorn.csv"

calc_outflow(inflowpath, delspath, evappath, savepath)

Plot the series

In [ ]:
from plotly.subplots import make_subplots
from datetime import datetime

def plot_reservoir(inflow_fn, outflow_fn, dels_fn, sarea_fn, reservoir_name):
    """Plots the fluxes of the reservoirs using `plotly`
    Args:
        inflow_fn (str): path of the inflow file
        outflow_fn (str): path of the outflow file
        dels_fn (str): path of the dels file
        sarea_fn (str): path of the sarea file
        reservoir_name (str): name of the reservoir. Will be used as title
        save_fn (str): path where to save the file
    """
    inflow = pd.read_csv(inflow_fn, parse_dates=['date'])   
    inflow['streamflow'] = inflow['streamflow'] *  (60*60*24) * 1e-6            # cumecs to million m3/d
    dels = pd.read_csv(dels_fn, parse_dates=['date'])[['date', 'dS']]
    dels['dS'] = dels['dS'] * 1e-6                                              # m3 to millions of m3

    last_date = dels['date'][:-1]
    dels = dels.iloc[1:,:]
    dels['last_date'] = last_date.values
    dels['days_passed'] = (dels['date'] - dels['last_date']).dt.days
    dels['dS_rate'] = dels['dS']/dels['days_passed']

    outflow = pd.read_csv(outflow_fn, parse_dates=['date'])[['date', 'outflow_rate']] 
    outflow['outflow_rate'] = outflow['outflow_rate'] * 1e-6                    # m3/d to millions of m3/d

    sarea = pd.read_csv(sarea_fn, parse_dates=['system:time_start']).rename({'system:time_start': 'date'}, axis=1)[['date', 'corrected_area']]

    # cleaning has to be done
    sarea.loc[sarea['corrected_area'] == -9999, 'corrected_area'] = np.nan
    sarea.loc[:, 'corrected_area'] = sarea.loc[:, 'corrected_area'].fillna(method='ffill')

    all_min = np.min([outflow.loc[:, 'outflow_rate'].min(), inflow.loc[:, 'streamflow'].min(), dels.loc[:, 'dS_rate'].min()])
    all_max = np.max([outflow.loc[:, 'outflow_rate'].max(), inflow.loc[:, 'streamflow'].max(), dels.loc[:, 'dS_rate'].max()])
    
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.3, 0.7],
        shared_xaxes=True,
        vertical_spacing=0.05
    )

    # Surface Area
    fig.add_trace(
        go.Scatter(
            x=list(sarea['date']),
            y=list(sarea['corrected_area']),
            name="Surface Area",
            showlegend=True,
            mode='markers+lines',
            hovertemplate=r'%{y:.3f}',
            line=dict(color='black'),
            yaxis='y'
        ),
        row=1, col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=list(inflow.date), 
            y=list(inflow.streamflow), 
            name="Inflow", 
            showlegend=True,
            hovertemplate=r'%{y:.3f}',
            line=dict(color='#4427B0'),
            # yaxis='y2'
        ),
        row=2, col=1,
    )

    # delta S
    fig.add_trace(
        go.Scatter(
            x=list(dels['date']),
            y=list(dels['dS_rate']),
            name="∆S",
            showlegend=True,
            mode='markers+lines',
            hovertemplate=r'%{y:.3f}',
            line=dict(color='#72C401'),
            # yaxis='y2',
        ),
        row=2, col=1
    )

    # Outflow
    fig.add_trace(
        go.Scatter(
            x=list(outflow['date']),
            y=list(outflow['outflow_rate']),
            name="Outflow",
            showlegend=True,
            mode='markers+lines',
            hovertemplate=r'%{y:.3f}',
            line=dict(color='#D81A6D'),
            # yaxis='y2'
        ),
        row=2, col=1,
    )

    # Update Layout
    fig['layout'].update(
        # Styling
        autosize=False, 
        height=400, 
        width=800,
        margin={
            'b': 10,
            'l': 10,
            'r': 10,
            't': 50
        },
        title=dict(
            text=reservoir_name,
            y=0.98,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        hovermode='x unified',
        xaxis1=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=6, label='6 Months', step="month", stepmode="backward"),
                    dict(count=1, label='1 Year', step="year", stepmode="backward"),
                    dict(count=1, label='This Year', step="year", stepmode="todate"),
                    dict(label='All', step="all")
                ])
            ),
        ),
        xaxis2=dict(
            type="date",
            rangeslider=dict(
                visible=True
            ),
            range=['2016-01-01', datetime.today().strftime('%Y-%m-%d')]
        ),
        yaxis=dict(
            anchor="x",
            title="Area (km²)"
        ),
        yaxis2=dict(
            anchor="x",
            title="Flux (×10⁶ m³/day)", #+ r"$\times 10^6 m^3/day$"
            # rangemode='normal',
            range=[all_min, all_max],
            scaleratio=1,
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            xanchor='right',
            y=1,
            x=1
        ),
    )
    fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor')

    return fig

In [ ]:
# Read in inflow
inflow = pd.read_csv(inflowpath, parse_dates=["date"])
inflow['streamflow'] = inflow['streamflow'] * (60*60*24)

# Read in ∆S
dels = pd.read_csv(delspath, parse_dates=["date"])

# Read in outflow
outflow = pd.read_csv(savepath, parse_dates=["date"])
outflow['outflow_rate']

plot_reservoir(inflowpath, savepath, delspath, "../../1_storage_change/data/area-sirindhorn.csv", "Sirindhorn")